Linear programming is used in supply chain optimization where the primary objective can be to minimize costs or maximizing profits, or reducing the delivery times especially when there are constraints such as limited resources, capacity or delivery deadlines.

So linear programming can be used in cases where there are
a) Transportation problems: to optimize shipping routes and costs

b) Production scheduling: determine what and how much to produce

c) Resource allocation: Assigning resources like machines.

There are three things that come into picture when we use linear programming to solve a problem.

1) Objective function: What is needed to be optimizzed? e.g., optimize cost

2) Decision variables: What are the variables that influence the objective function? e.g., quantitiy to produce.

3) Constraints: What are the limitations or rules that the solution must satisfy? e.g., demand fulfillment.



In [1]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 31.1 MB/s eta 0:00:00


In [2]:
import pulp ##package to specifically solve linear programming in python

We are solving the problem of demand and supply.
We have 4 supply plants: Dallas, Los Angeles, Seattle, Denver.
We have to find the number of products that these plants can supply annually.

for this rememeber that we have to also consider the fixed cost of building each supply plant.

Demand part: 4 stores : Austin, Sacramento, Philadelphia, Boulder.

Now the demand is that Austin requires 1700 units each year.

This is the supply part

In [3]:
plants = ['Dallas', 'Los Angeles', 'Seattle', 'Denver']

In [4]:
#annual supply these plants can ship
supply = {'Dallas' : 900, 'Los Angeles' : 2400, 'Seattle' : 1300, 'Denver' : 1800}

In [5]:
##fixed cost of building each plant is to be considered.
fixedCost = {'Dallas' : 75000, 'Los Angeles' : 72000, 'Seattle' : 100000, 'Denver' : 74000}

This is the Demand part

In [6]:
stores = ['Austin' , 'Sacramento', 'Philadelphia', 'Boulder']

In [7]:
##this is the demand from each store.
demand = {'Austin' : 1700, 'Sacramento' : 1000, 'Philadelphia' : 1500, 'Boulder' : 1200}

In [8]:
##Now, we need to look at the shipping charges for each product from one plant to one store.
## Thus we create a cost matrix for this

In [9]:
cost_matrix = [
  #atx #sct #phl #bu
  [2, 7, 4, 6],#dal
  [6, 3, 4, 5],#la
  [8, 4, 6, 5],#sea
  [7, 6, 5, 1], #den
]

In [10]:
cost_matrix

[[2, 7, 4, 6], [6, 3, 4, 5], [8, 4, 6, 5], [7, 6, 5, 1]]

In [11]:
### we have list of lists but we need dictionaries.

costs = {}
for i in range(len(plants)):
  temp_dict = {}
  for j in range(len(stores)):
    temp_dict[stores[j]] = cost_matrix[i][j]
  costs[plants[i]] = temp_dict

In [12]:
costs

{'Dallas': {'Austin': 2, 'Sacramento': 7, 'Philadelphia': 4, 'Boulder': 6},
 'Los Angeles': {'Austin': 6,
  'Sacramento': 3,
  'Philadelphia': 4,
  'Boulder': 5},
 'Seattle': {'Austin': 8, 'Sacramento': 4, 'Philadelphia': 6, 'Boulder': 5},
 'Denver': {'Austin': 7, 'Sacramento': 6, 'Philadelphia': 5, 'Boulder': 1}}

In [13]:
routes = []
for p in plants:
  for s in stores:
    routes.append((p,s))

In [14]:
routes

[('Dallas', 'Austin'),
 ('Dallas', 'Sacramento'),
 ('Dallas', 'Philadelphia'),
 ('Dallas', 'Boulder'),
 ('Los Angeles', 'Austin'),
 ('Los Angeles', 'Sacramento'),
 ('Los Angeles', 'Philadelphia'),
 ('Los Angeles', 'Boulder'),
 ('Seattle', 'Austin'),
 ('Seattle', 'Sacramento'),
 ('Seattle', 'Philadelphia'),
 ('Seattle', 'Boulder'),
 ('Denver', 'Austin'),
 ('Denver', 'Sacramento'),
 ('Denver', 'Philadelphia'),
 ('Denver', 'Boulder')]

In [15]:
##use pulp now

route = pulp.LpVariable.dicts('Route', (plants, stores), 0, None, pulp.LpInteger)

In [16]:
route

{'Dallas': {'Austin': Route_Dallas_Austin,
  'Sacramento': Route_Dallas_Sacramento,
  'Philadelphia': Route_Dallas_Philadelphia,
  'Boulder': Route_Dallas_Boulder},
 'Los Angeles': {'Austin': Route_Los_Angeles_Austin,
  'Sacramento': Route_Los_Angeles_Sacramento,
  'Philadelphia': Route_Los_Angeles_Philadelphia,
  'Boulder': Route_Los_Angeles_Boulder},
 'Seattle': {'Austin': Route_Seattle_Austin,
  'Sacramento': Route_Seattle_Sacramento,
  'Philadelphia': Route_Seattle_Philadelphia,
  'Boulder': Route_Seattle_Boulder},
 'Denver': {'Austin': Route_Denver_Austin,
  'Sacramento': Route_Denver_Sacramento,
  'Philadelphia': Route_Denver_Philadelphia,
  'Boulder': Route_Denver_Boulder}}

In [17]:
## pulp.LpVariable.dicts(name, indexes, lowerbound, upperbound, category)

In [18]:
build = pulp.LpVariable.dicts("Build_Plant", plants, 0, 1, pulp.LpInteger)

In [19]:
build

{'Dallas': Build_Plant_Dallas,
 'Los Angeles': Build_Plant_Los_Angeles,
 'Seattle': Build_Plant_Seattle,
 'Denver': Build_Plant_Denver}

In [20]:
## Defining the objective function is important
## this is going to be a multiplication of number of units traveling along each route * cost of shipping a unit in that route

obj = ""

for (p,s) in routes:
  obj += route[p][s] * costs[p][s]

obj

2*Route_Dallas_Austin + 6*Route_Dallas_Boulder + 4*Route_Dallas_Philadelphia + 7*Route_Dallas_Sacramento + 7*Route_Denver_Austin + 1*Route_Denver_Boulder + 5*Route_Denver_Philadelphia + 6*Route_Denver_Sacramento + 6*Route_Los_Angeles_Austin + 5*Route_Los_Angeles_Boulder + 4*Route_Los_Angeles_Philadelphia + 3*Route_Los_Angeles_Sacramento + 8*Route_Seattle_Austin + 5*Route_Seattle_Boulder + 6*Route_Seattle_Philadelphia + 4*Route_Seattle_Sacramento + 0

In [21]:
for p in plants:
  obj += fixedCost[p] * build[p]
obj

75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seattle + 2*Route_Dallas_Austin + 6*Route_Dallas_Boulder + 4*Route_Dallas_Philadelphia + 7*Route_Dallas_Sacramento + 7*Route_Denver_Austin + 1*Route_Denver_Boulder + 5*Route_Denver_Philadelphia + 6*Route_Denver_Sacramento + 6*Route_Los_Angeles_Austin + 5*Route_Los_Angeles_Boulder + 4*Route_Los_Angeles_Philadelphia + 3*Route_Los_Angeles_Sacramento + 8*Route_Seattle_Austin + 5*Route_Seattle_Boulder + 6*Route_Seattle_Philadelphia + 4*Route_Seattle_Sacramento + 0

In [22]:
##problem statement

prob = pulp.LpProblem("Supply_Problem", pulp.LpMinimize)
prob += obj, "TOTAL COST"
prob

Supply_Problem:
MINIMIZE
75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seattle + 2*Route_Dallas_Austin + 6*Route_Dallas_Boulder + 4*Route_Dallas_Philadelphia + 7*Route_Dallas_Sacramento + 7*Route_Denver_Austin + 1*Route_Denver_Boulder + 5*Route_Denver_Philadelphia + 6*Route_Denver_Sacramento + 6*Route_Los_Angeles_Austin + 5*Route_Los_Angeles_Boulder + 4*Route_Los_Angeles_Philadelphia + 3*Route_Los_Angeles_Sacramento + 8*Route_Seattle_Austin + 5*Route_Seattle_Boulder + 6*Route_Seattle_Philadelphia + 4*Route_Seattle_Sacramento + 0
VARIABLES
0 <= Build_Plant_Dallas <= 1 Integer
0 <= Build_Plant_Denver <= 1 Integer
0 <= Build_Plant_Los_Angeles <= 1 Integer
0 <= Build_Plant_Seattle <= 1 Integer
0 <= Route_Dallas_Austin Integer
0 <= Route_Dallas_Boulder Integer
0 <= Route_Dallas_Philadelphia Integer
0 <= Route_Dallas_Sacramento Integer
0 <= Route_Denver_Austin Integer
0 <= Route_Denver_Boulder Integer
0 <= Route_Denver_Philadelphia I

In [23]:
##supply/demand constraints

## 1) Supply constrant: Number of products that can be shipped out of each plant
for p in plants:
  product_out = ""
  for s in stores:
    product_out += route[p][s]
  prob += product_out <= supply[p] * build[p], "Total product out of plant" + str(p)
prob

Supply_Problem:
MINIMIZE
75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seattle + 2*Route_Dallas_Austin + 6*Route_Dallas_Boulder + 4*Route_Dallas_Philadelphia + 7*Route_Dallas_Sacramento + 7*Route_Denver_Austin + 1*Route_Denver_Boulder + 5*Route_Denver_Philadelphia + 6*Route_Denver_Sacramento + 6*Route_Los_Angeles_Austin + 5*Route_Los_Angeles_Boulder + 4*Route_Los_Angeles_Philadelphia + 3*Route_Los_Angeles_Sacramento + 8*Route_Seattle_Austin + 5*Route_Seattle_Boulder + 6*Route_Seattle_Philadelphia + 4*Route_Seattle_Sacramento + 0
SUBJECT TO
Total_product_out_of_plantDallas: - 900 Build_Plant_Dallas
 + Route_Dallas_Austin + Route_Dallas_Boulder + Route_Dallas_Philadelphia
 + Route_Dallas_Sacramento <= 0

Total_product_out_of_plantLos_Angeles: - 2400 Build_Plant_Los_Angeles
 + Route_Los_Angeles_Austin + Route_Los_Angeles_Boulder
 + Route_Los_Angeles_Philadelphia + Route_Los_Angeles_Sacramento <= 0

Total_product_out_of_plantSeattl

In [24]:
###Demand constraint
##For each stores, total products that are coming from the plants is greater than or equal to what
##is required.

for s in stores:
  product_in = ""
  for p in plants:
    product_in += route[p][s]
  prob += product_in >= demand[s], "Total product into the store" + str(s)
prob

Supply_Problem:
MINIMIZE
75000*Build_Plant_Dallas + 74000*Build_Plant_Denver + 72000*Build_Plant_Los_Angeles + 100000*Build_Plant_Seattle + 2*Route_Dallas_Austin + 6*Route_Dallas_Boulder + 4*Route_Dallas_Philadelphia + 7*Route_Dallas_Sacramento + 7*Route_Denver_Austin + 1*Route_Denver_Boulder + 5*Route_Denver_Philadelphia + 6*Route_Denver_Sacramento + 6*Route_Los_Angeles_Austin + 5*Route_Los_Angeles_Boulder + 4*Route_Los_Angeles_Philadelphia + 3*Route_Los_Angeles_Sacramento + 8*Route_Seattle_Austin + 5*Route_Seattle_Boulder + 6*Route_Seattle_Philadelphia + 4*Route_Seattle_Sacramento + 0
SUBJECT TO
Total_product_out_of_plantDallas: - 900 Build_Plant_Dallas
 + Route_Dallas_Austin + Route_Dallas_Boulder + Route_Dallas_Philadelphia
 + Route_Dallas_Sacramento <= 0

Total_product_out_of_plantLos_Angeles: - 2400 Build_Plant_Los_Angeles
 + Route_Los_Angeles_Austin + Route_Los_Angeles_Boulder
 + Route_Los_Angeles_Philadelphia + Route_Los_Angeles_Sacramento <= 0

Total_product_out_of_plantSeattl

In [25]:
##Solve here:
prob.solve()

1

In [26]:
print("Status:", pulp.LpStatus[prob.status])

Status: Optimal


In [27]:
pulp.LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

In [28]:
## To determine which plant is to be build.
for v in prob.variables():
  print(v.name, "=", v.varValue)

Build_Plant_Dallas = 0.0
Build_Plant_Denver = 1.0
Build_Plant_Los_Angeles = 1.0
Build_Plant_Seattle = 1.0
Route_Dallas_Austin = 0.0
Route_Dallas_Boulder = 0.0
Route_Dallas_Philadelphia = 0.0
Route_Dallas_Sacramento = 0.0
Route_Denver_Austin = 600.0
Route_Denver_Boulder = 1200.0
Route_Denver_Philadelphia = 0.0
Route_Denver_Sacramento = 0.0
Route_Los_Angeles_Austin = 900.0
Route_Los_Angeles_Boulder = 0.0
Route_Los_Angeles_Philadelphia = 1500.0
Route_Los_Angeles_Sacramento = 0.0
Route_Seattle_Austin = 200.0
Route_Seattle_Boulder = 0.0
Route_Seattle_Philadelphia = 0.0
Route_Seattle_Sacramento = 1000.0


In [29]:
##here we can see that we should build all the plants except for the plant at dallas.

In [30]:
pulp.value(prob.objective)

268400.0

In [31]:
##How much should be send to each store from each plant?
for p in plants:
    for s in stores:
        print(f"Ship {route[p][s].varValue} units from {p} to {s}")

Ship 0.0 units from Dallas to Austin
Ship 0.0 units from Dallas to Sacramento
Ship 0.0 units from Dallas to Philadelphia
Ship 0.0 units from Dallas to Boulder
Ship 900.0 units from Los Angeles to Austin
Ship 0.0 units from Los Angeles to Sacramento
Ship 1500.0 units from Los Angeles to Philadelphia
Ship 0.0 units from Los Angeles to Boulder
Ship 200.0 units from Seattle to Austin
Ship 1000.0 units from Seattle to Sacramento
Ship 0.0 units from Seattle to Philadelphia
Ship 0.0 units from Seattle to Boulder
Ship 600.0 units from Denver to Austin
Ship 0.0 units from Denver to Sacramento
Ship 0.0 units from Denver to Philadelphia
Ship 1200.0 units from Denver to Boulder


In [32]:
##how much money is required to build a plant
fixed_cost_total = sum(fixedCost[p] * build[p].varValue for p in plants)
print(f"Total Fixed Costs: ${fixed_cost_total}")


Total Fixed Costs: $246000.0
